In [ ]:
!kaggle datasets download -d bwandowando/asean-cities-weather-data-2010-2024

Dataset URL: https://www.kaggle.com/datasets/bwandowando/asean-cities-weather-data-2010-2024
License(s): Attribution 4.0 International (CC BY 4.0)
 99% 506M/511M [00:09<00:00, 60.3MB/s]
100% 511M/511M [00:09<00:00, 57.0MB/s]


In [ ]:
!unzip asean-cities-weather-data-2010-2024.zip

Archive:  asean-cities-weather-data-2010-2024.zip
  inflating: ASEAN_cities.csv        
  inflating: daily_data_combined_2010_to_present.csv  
  inflating: daily_units_1980_to_present.csv  
  inflating: hour_units_1980_to_present.csv  
  inflating: hourly_data_combined_2010_to_present.csv  


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_log_error
print('The scikit-learn version is {}.'.format(sklearn.__version__))
plt.style.use('ggplot')

The scikit-learn version is 1.5.2.


In [ ]:
df_csv = pd.read_csv('/content/ASEAN_cities.csv')
df_csv[df_csv['country']=='Indonesia']

In [ ]:
df_hourly = pd.read_csv('/content/hourly_data_combined_2010_to_present.csv')
df_hourly = df_hourly.loc[df_hourly['city_name'].isin(['Jakarta','Surabaya','Semarang'])]
df_hourly

In [ ]:
import pandas as pd

# Asumsikan df_hourly sudah di-load dan kolom 'datetime' bertipe datetime
df_hourly['datetime'] = pd.to_datetime(df_hourly['datetime'])  # Pastikan kolom datetime dalam format datetime

# Filter berdasarkan tahun 2019-2024
df_hourly = df_hourly[(df_hourly['datetime'].dt.year >= 2019) & (df_hourly['datetime'].dt.year <= 2023)]

# Filter berdasarkan jam tertentu
hours_to_select = [1, 4, 7, 10, 13, 16, 19, 22]
df_filtered = df_hourly[df_hourly['datetime'].dt.hour.isin(hours_to_select)]

# Pilih hanya kolom yang relevan
df_filtered = df_filtered[['city_name', 'datetime', 'temperature_2m', 'relative_humidity_2m']]

# Ambil hanya tanggal (tanpa jam) di kolom 'datetime'
# df_filtered['date'] = df_filtered['datetime'].dt.date

# Tampilkan hasil
df_filtered.head(100)


,city_name,datetime,temperature_2m,relative_humidity_2m
3332689,Jakarta,2019-01-01 01:00:00,24.7,91.0
3332692,Jakarta,2019-01-01 04:00:00,25.9,87.0
3332695,Jakarta,2019-01-01 07:00:00,26.0,91.0
3332698,Jakarta,2019-01-01 10:00:00,25.8,89.0
3332701,Jakarta,2019-01-01 13:00:00,25.4,86.0
...,...,...,...,...
3332974,Jakarta,2019-01-12 22:00:00,25.0,94.0
3332977,Jakarta,2019-01-13 01:00:00,27.0,85.0
3332980,Jakarta,2019-01-13 04:00:00,28.2,81.0
3332983,Jakarta,2019-01-13 07:00:00,28.8,78.0


In [ ]:
import pandas as pd

# Asumsikan df_filtered sudah di-load dan sudah difilter berdasarkan tahun dan jam yang relevan

# Ambil hanya tanggal dan jam untuk digunakan sebagai kolom
df_filtered['date'] = df_filtered['datetime'].dt.date
df_filtered['hour'] = df_filtered['datetime'].dt.hour

# Pivot tabel untuk suhu (temperature_2m)
df_temperature = df_filtered.pivot_table(index=['city_name', 'date'], columns='hour', values='temperature_2m', aggfunc='first')

# Ubah nama kolom untuk suhu menjadi T1, T2, ..., T24
df_temperature.columns = [f'T{col+1}' for col in df_temperature.columns]

# Pivot tabel untuk kelembapan relatif (relative_humidity_2m)
df_humidity = df_filtered.pivot_table(index=['city_name', 'date'], columns='hour', values='relative_humidity_2m', aggfunc='first')

# Ubah nama kolom untuk kelembapan menjadi H1, H2, ..., H24
df_humidity.columns = [f'H{col+1}' for col in df_humidity.columns]

# Gabungkan kedua DataFrame (temperature dan humidity) berdasarkan city_name dan date
df_pivoted = pd.concat([df_temperature, df_humidity], axis=1)

# Reset index untuk mendapatkan format DataFrame biasa
df_pivoted = df_pivoted.reset_index()

# Menampilkan hasil
print(df_pivoted)


     city_name        date    T2    T5    T8   T11   T14   T17   T20   T23  \
0      Jakarta  2019-01-01  24.7  25.9  26.0  25.8  25.4  25.2  24.6  24.4   
1      Jakarta  2019-01-02  25.3  28.7  29.6  27.9  26.2  25.5  24.9  24.7   
2      Jakarta  2019-01-03  24.3  26.4  30.1  28.3  27.2  25.1  24.0  23.7   
3      Jakarta  2019-01-04  27.1  31.1  30.6  29.7  27.9  26.7  25.3  24.4   
4      Jakarta  2019-01-05  27.3  30.7  29.9  28.8  27.4  26.5  25.4  24.8   
...        ...         ...   ...   ...   ...   ...   ...   ...   ...   ...   
5473  Surabaya  2023-12-27  30.0  33.3  31.8  30.4  27.3  26.7  25.4  25.2   
5474  Surabaya  2023-12-28  29.6  32.4  33.3  29.4  27.6  26.1  25.4  25.7   
5475  Surabaya  2023-12-29  29.9  33.7  34.3  29.8  28.7  27.1  25.0  24.8   
5476  Surabaya  2023-12-30  29.3  31.8  31.4  27.9  28.8  25.5  25.4  24.4   
5477  Surabaya  2023-12-31  30.1  33.9  34.2  29.9  28.1  27.3  26.0  25.1   

        H2    H5    H8   H11   H14   H17   H20   H23  
0     91

In [ ]:
# Hitung rolling average untuk suhu berdasarkan kolom T1, T2, ..., T24 (dengan jendela 3 jam)
df_temperature_avg = df_pivoted[[ 'T2','T5','T8','T11','T14','T17','T20','T23' ]].rolling(window=3, axis=1).mean()

# Buat kolom 'target_temperature' berdasarkan rata-rata rolling untuk suhu
df_pivoted['target_temperature'] = df_temperature_avg.mean(axis=1).round(1)

# Hitung rolling average untuk kelembapan berdasarkan kolom H1, H2, ..., H24 (dengan jendela 3 jam)
df_humidity_avg = df_pivoted[[ 'H2','H5','H8','H11','H14','H17','H20','H23']].rolling(window=3, axis=1).mean()

# Buat kolom 'target_humidity' berdasarkan rata-rata rolling untuk kelembapan
df_pivoted['target_humidity'] = df_humidity_avg.mean(axis=1).round(1)

# Menampilkan hasil
print(df_pivoted.head(100))


<ipython-input-7-5fe42add5872>:2: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
  df_temperature_avg = df_pivoted[[ 'T2','T5','T8','T11','T14','T17','T20','T23' ]].rolling(window=3, axis=1).mean()
<ipython-input-7-5fe42add5872>:8: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
  df_humidity_avg = df_pivoted[[ 'H2','H5','H8','H11','H14','H17','H20','H23']].rolling(window=3, axis=1).mean()


   city_name        date    T2    T5    T8   T11   T14   T17   T20   T23  \
0    Jakarta  2019-01-01  24.7  25.9  26.0  25.8  25.4  25.2  24.6  24.4   
1    Jakarta  2019-01-02  25.3  28.7  29.6  27.9  26.2  25.5  24.9  24.7   
2    Jakarta  2019-01-03  24.3  26.4  30.1  28.3  27.2  25.1  24.0  23.7   
3    Jakarta  2019-01-04  27.1  31.1  30.6  29.7  27.9  26.7  25.3  24.4   
4    Jakarta  2019-01-05  27.3  30.7  29.9  28.8  27.4  26.5  25.4  24.8   
..       ...         ...   ...   ...   ...   ...   ...   ...   ...   ...   
95   Jakarta  2019-04-06  26.9  30.7  30.1  28.4  27.1  25.8  25.0  24.7   
96   Jakarta  2019-04-07  27.7  30.5  30.0  27.0  25.7  25.7  25.2  24.8   
97   Jakarta  2019-04-08  26.6  30.2  30.7  26.9  25.8  25.2  24.7  24.3   
98   Jakarta  2019-04-09  26.9  30.3  29.5  28.6  27.7  27.1  25.7  24.9   
99   Jakarta  2019-04-10  27.5  30.3  30.2  28.1  27.9  26.9  25.6  24.7   

      H2    H5    H8   H11   H14   H17   H20   H23  target_temperature  \
0   91.0  87.

In [ ]:
!kaggle datasets download -d nikhil7280/weather-type-classification

Dataset URL: https://www.kaggle.com/datasets/nikhil7280/weather-type-classification
License(s): other
  0% 0.00/186k [00:00<?, ?B/s]
100% 186k/186k [00:00<00:00, 86.6MB/s]


In [ ]:
!unzip weather-type-classification.zip

Archive:  weather-type-classification.zip
  inflating: weather_classification_data.csv  


In [ ]:
import pandas as pd

In [ ]:
df_weather = pd.read_csv('/content/weather_classification_data.csv')
df_weather

In [ ]:
# prompt: Using dataframe df_weather: drop row some spesific lower number on column

# Assuming you want to drop rows where 'Temperature' is below a certain value.
# Replace 0 with the desired lower bound.

lower_bound = 10  # Example: remove rows with temperature below 0 degrees.

# Drop rows where 'Temperature' is below the specified lower bound.
df = df_weather[df_weather['Temperature'] >= lower_bound]


In [ ]:
df_weather = df_weather[(df_weather['Season'] != 'Winter') &(df_weather['Weather Type'] != 'Snowy')  & ((df_weather['Location'] != 'mountain') & (df_weather['Location'] != 'coastal'))]
# df_weather

In [ ]:
print(df_weather['Cloud Cover'].value_counts())

Cloud Cover
partly cloudy    963
overcast         840
clear            516
cloudy            66
Name: count, dtype: int64


In [ ]:
df_weather.loc[(df_weather['Weather Type'] == 'Rainy')].head(100)

In [ ]:
df_weather.loc[(df_weather['Cloud Cover'] == 'partly cloudy') & (df_weather['Weather Type'] == 'Sunny'), 'Weather Type'] = 'partly cloudy'
df_weather.loc[(df_weather['Cloud Cover'] == 'overcast') & (df_weather['Weather Type'] == 'Sunny'), 'Weather Type'] = 'Isolated Shower'
df_weather.loc[(df_weather['Cloud Cover'] == 'partly cloudy') & (df_weather['Weather Type'] == 'Cloudy'), 'Weather Type'] = 'Mostly Cloudy'
df_weather.loc[(df_weather['Cloud Cover'] == 'cloudy') & (df_weather['Weather Type'] == 'Cloudy'), 'Weather Type'] = 'Mostly Cloudy'
df_weather.loc[(df_weather['Cloud Cover'] == 'overcast') & (df_weather['Weather Type'] == 'Cloudy'), 'Weather Type'] = 'overcast'
df_weather.loc[(df_weather['Cloud Cover'] == 'cloudy') & (df_weather['Weather Type'] == 'Rainy'), 'Weather Type'] = 'Light Rain'
df_weather.loc[(df_weather['Cloud Cover'] == 'partly cloudy') & (df_weather['Weather Type'] == 'Rainy'), 'Weather Type'] = 'Light Rain'


print(df_weather['Weather Type'].value_counts())

Weather Type
Sunny              541
Mostly Cloudy      527
Rainy              527
overcast           284
Light Rain         255
partly cloudy      222
Isolated Shower     29
Name: count, dtype: int64


In [ ]:
df_weather.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
1,39.0,96,8.5,71.0,partly cloudy,1011.43,7,Spring,10.0,inland,Mostly Cloudy
5,32.0,55,3.5,26.0,overcast,1010.03,2,Summer,5.0,inland,overcast
11,38.0,43,2.0,16.0,clear,1029.16,11,Autumn,7.5,inland,Sunny
16,30.0,27,7.0,13.0,partly cloudy,1016.38,5,Spring,7.5,inland,partly cloudy
19,13.0,102,12.0,72.0,clear,1012.25,4,Summer,8.0,inland,Sunny


In [ ]:
# Mapping nilai lama ke nilai baru
weather_mapping = {
    'Sunny': 'Cerah',
    'Mostly Cloudy': 'Berawan',
    'Rainy': 'Hujan_Sedang',
    'overcast': 'Berawan_Tebal',
    'Light Rain': 'Hujan_Ringan',
    'partly cloudy': 'Cerah_Berawan',
    'Isolated Shower': 'Hujan_Lokal'
}

# Mengganti nilai di kolom 'Weather Type'
df_weather['Weather Type'] = df_weather['Weather Type'].replace(weather_mapping)

# Menampilkan hasil
print(df_weather['Weather Type'].value_counts())


Weather Type
Cerah            541
Berawan          527
Hujan_Sedang     527
Berawan_Tebal    284
Hujan_Ringan     255
Cerah_Berawan    222
Hujan_Lokal       29
Name: count, dtype: int64


<ipython-input-17-f76195ce625f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weather['Weather Type'] = df_weather['Weather Type'].replace(weather_mapping)


In [ ]:
df = df_pivoted

In [ ]:
df['date'] = pd.to_datetime(df['date'])

# Buat kolom baru dengan format MM-DD
df['month_day'] = df['date'].dt.strftime('%m-%d')

# Menampilkan hasi
df

,city_name,date,T2,T5,T8,T11,T14,T17,T20,T23,...,H5,H8,H11,H14,H17,H20,H23,target_temperature,target_humidity,month_day
0,Jakarta,2019-01-01,24.7,25.9,26.0,25.8,25.4,25.2,24.6,24.4,...,87.0,91.0,89.0,86.0,86.0,88.0,88.0,25.4,88.1,01-01
1,Jakarta,2019-01-02,25.3,28.7,29.6,27.9,26.2,25.5,24.9,24.7,...,72.0,72.0,75.0,84.0,85.0,87.0,87.0,26.9,80.1,01-02
2,Jakarta,2019-01-03,24.3,26.4,30.1,28.3,27.2,25.1,24.0,23.7,...,86.0,64.0,71.0,69.0,80.0,86.0,87.0,26.7,76.3,01-03
3,Jakarta,2019-01-04,27.1,31.1,30.6,29.7,27.9,26.7,25.3,24.4,...,54.0,60.0,66.0,68.0,73.0,81.0,87.0,28.3,68.4,01-04
4,Jakarta,2019-01-05,27.3,30.7,29.9,28.8,27.4,26.5,25.4,24.8,...,58.0,67.0,73.0,83.0,90.0,95.0,97.0,27.9,78.6,01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5473,Surabaya,2023-12-27,30.0,33.3,31.8,30.4,27.3,26.7,25.4,25.2,...,59.0,61.0,66.0,87.0,92.0,98.0,96.0,29.0,77.7,12-27
5474,Surabaya,2023-12-28,29.6,32.4,33.3,29.4,27.6,26.1,25.4,25.7,...,62.0,53.0,74.0,88.0,94.0,95.0,95.0,28.9,78.4,12-28
5475,Surabaya,2023-12-29,29.9,33.7,34.3,29.8,28.7,27.1,25.0,24.8,...,60.0,49.0,72.0,82.0,92.0,96.0,96.0,29.5,76.1,12-29
5476,Surabaya,2023-12-30,29.3,31.8,31.4,27.9,28.8,25.5,25.4,24.4,...,67.0,67.0,79.0,85.0,93.0,94.0,99.0,28.3,81.7,12-30


In [ ]:
import pandas as pd
import numpy as np

# Asumsikan df dan df_weather sudah di-load

# Fungsi untuk mencari indeks dengan nilai terdekat
def find_closest_weather(row, df_weather):
    # Hitung perbedaan absolut untuk Temperature dan Humidity
    df_weather['temp_diff'] = abs(df_weather['Temperature'] - row['target_temperature'])
    df_weather['humidity_diff'] = abs(df_weather['Humidity'] - row['target_humidity'])

    # Totalkan perbedaan untuk menemukan pasangan terdekat
    df_weather['total_diff'] = df_weather['temp_diff'] + df_weather['humidity_diff']

    # Cari baris dengan perbedaan total terkecil
    closest_row = df_weather.loc[df_weather['total_diff'].idxmin()]

    # Kembalikan Weather Type dari baris terdekat
    return closest_row['Weather Type']

# Terapkan fungsi ke setiap baris di df
df['klasifikasi'] = df.apply(find_closest_weather, axis=1, df_weather=df_weather)

# Drop kolom tambahan di df_weather
df_weather.drop(['temp_diff', 'humidity_diff', 'total_diff'], axis=1, inplace=True)

# Menampilkan hasil
print(df.head())


  city_name       date    T2    T5    T8   T11   T14   T17   T20   T23  ...  \
0   Jakarta 2019-01-01  24.7  25.9  26.0  25.8  25.4  25.2  24.6  24.4  ...   
1   Jakarta 2019-01-02  25.3  28.7  29.6  27.9  26.2  25.5  24.9  24.7  ...   
2   Jakarta 2019-01-03  24.3  26.4  30.1  28.3  27.2  25.1  24.0  23.7  ...   
3   Jakarta 2019-01-04  27.1  31.1  30.6  29.7  27.9  26.7  25.3  24.4  ...   
4   Jakarta 2019-01-05  27.3  30.7  29.9  28.8  27.4  26.5  25.4  24.8  ...   

     H8   H11   H14   H17   H20   H23  target_temperature  target_humidity  \
0  91.0  89.0  86.0  86.0  88.0  88.0                25.4             88.1   
1  72.0  75.0  84.0  85.0  87.0  87.0                26.9             80.1   
2  64.0  71.0  69.0  80.0  86.0  87.0                26.7             76.3   
3  60.0  66.0  68.0  73.0  81.0  87.0                28.3             68.4   
4  67.0  73.0  83.0  90.0  95.0  97.0                27.9             78.6   

   month_day    klasifikasi  
0      01-01   Hujan_Sedan

<ipython-input-20-959a4269a967>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weather.drop(['temp_diff', 'humidity_diff', 'total_diff'], axis=1, inplace=True)


In [ ]:
selected_columns = ['target_temperature','target_humidity','klasifikasi','month_day']
df_selected = df[selected_columns]
df_selected

,target_temperature,target_humidity,klasifikasi,month_day
0,25.4,88.1,Hujan Sedang,01-01
1,26.9,80.1,Hujan Sedang,01-02
2,26.7,76.3,Hujan Ringan,01-03
3,28.3,68.4,Berawan Tebal,01-04
4,27.9,78.6,Hujan Sedang,01-05
...,...,...,...,...
5473,29.0,77.7,Berawan,12-27
5474,28.9,78.4,Berawan,12-28
5475,29.5,76.1,Berawan,12-29
5476,28.3,81.7,Hujan Sedang,12-30


In [ ]:
df = df_selected
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5478 entries, 0 to 5477
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   target_temperature  5478 non-null   float64
 1   target_humidity     5478 non-null   float64
 2   klasifikasi         5478 non-null   object 
 3   month_day           5478 non-null   object 
dtypes: float64(2), object(2)
memory usage: 171.3+ KB


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [ ]:
from sklearn.model_selection import train_test_split

X = df[['target_temperature', 'target_humidity']]
y = df['klasifikasi']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Inisialisasi dan pelatihan model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9662408759124088
Classification Report:
                precision    recall  f1-score   support

      Berawan       0.96      0.97      0.96       306
Berawan_Tebal       0.94      0.92      0.93        91
        Cerah       0.80      0.91      0.85        35
Cerah_Berawan       0.86      0.75      0.80         8
  Hujan_Lokal       0.00      0.00      0.00         1
 Hujan_Ringan       0.99      0.97      0.98       168
 Hujan_Sedang       0.98      0.98      0.98       487

     accuracy                           0.97      1096
    macro avg       0.79      0.79      0.79      1096
 weighted avg       0.97      0.97      0.97      1096



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
new_data = [[27, 89]]
predicted_class = model.predict(new_data)
print("Predicted Class:", predicted_class[0])

Predicted Class: Hujan_Sedang


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [ ]:
import joblib

# Simpan model
joblib.dump(model, 'weather_prediction_model.pkl')

['weather_prediction_model.pkl']

In [ ]:
import pandas as pd


In [ ]:
wisata = pd.read_csv('/content/3-datatourism-fixed.csv')
wisata.head()

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Place_Ratings,Rekomen Cuaca
0,1.0,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000.0,4.6,3.722222,Hujan
1,2.0,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0.0,4.6,2.840000,Cerah
2,3.0,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000.0,4.6,2.526316,Cerah
3,4.0,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000.0,4.5,2.857143,Cerah
4,5.0,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000.0,4.5,3.520000,Cerah


In [ ]:
# prompt: get all value of city on variable wisata

unique_cities = wisata['City'].unique()
unique_cities

array(['Jakarta', 'Yogyakarta', 'Lainnya', 'Bandung', 'Semarang',
       'Surabaya'], dtype=object)

In [ ]:
# prompt: see city value 'Lainnya'

# Assuming 'wisata' DataFrame is already loaded as shown in the provided code.

city_counts = wisata['City'].value_counts()
print(city_counts)

# Check if 'Lainnya' is present in the 'City' column
if 'Lainnya' in wisata['City'].values:
  print("The city value 'Lainnya' exists in the DataFrame.")
  # Access rows where 'City' is 'Lainnya'
  lainnya_rows = wisata[wisata['City'] == 'Lainnya']
  print(lainnya_rows)
else:
  print("The city value 'Lainnya' does not exist in the DataFrame.")

City
Yogyakarta    125
Bandung       122
Jakarta        84
Semarang       57
Surabaya       44
Name: count, dtype: int64
The city value 'Lainnya' does not exist in the DataFrame.


In [ ]:
# prompt: create new column and the name is "kode"

kode_mapping = {
    'Yogyakarta': "18.10.02.2012",
    'Bandung': "32.73.17.1003",
    'Jakarta': "31.71.03.1001",
    'Semarang': "33.74.01.1009",
    'Surabaya ': "35.78.02.1003"
}
wisata['kode'] = wisata['City'].map(kode_mapping)

In [ ]:
wisata.head()

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Place_Ratings,Rekomen Cuaca,kode
0,1.0,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000.0,4.6,3.722222,Hujan,31.71.03.1001
1,2.0,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0.0,4.6,2.840000,Cerah,31.71.03.1001
2,3.0,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000.0,4.6,2.526316,Cerah,31.71.03.1001
3,4.0,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000.0,4.5,2.857143,Cerah,31.71.03.1001
4,5.0,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000.0,4.5,3.520000,Cerah,31.71.03.1001


In [ ]:
# prompt: drop column Rekomen Cuaca	on wisata

wisata = wisata.drop('Rekomen Cuaca', axis=1)
wisata.head()

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Place_Ratings,kode
0,1.0,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000.0,4.6,3.722222,31.71.03.1001
1,2.0,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0.0,4.6,2.840000,31.71.03.1001
2,3.0,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000.0,4.6,2.526316,31.71.03.1001
3,4.0,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000.0,4.5,2.857143,31.71.03.1001
4,5.0,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000.0,4.5,3.520000,31.71.03.1001


In [ ]:
# prompt: save wisata to csv

wisata.to_csv('wisata.csv', index=False)

In [ ]:
df_wisata = pd.read_csv('/content/wisata.csv')
df_wisata

In [ ]:
# prompt: convert csv file  to  sql file

import pandas as pd

# Load the CSV file into a pandas DataFrame
df_wisata = pd.read_csv('/content/wisata.csv')

# Create a SQL INSERT statement for each row in the DataFrame
sql_statements = []
for index, row in df_wisata.iterrows():
    values = ", ".join([f"'{str(x)}'" for x in row.values])  # Escape single quotes within values
    sql_statements.append(f"INSERT INTO wisata VALUES ({values});")


# Create a SQL file
with open('wisata.sql', 'w') as f:
    # Write the CREATE TABLE statement (adjust column names and data types)
    f.write("CREATE TABLE wisata (\n")
    f.write("    City VARCHAR(255),\n")  # Example column definition, modify as needed
    f.write("    Wisata VARCHAR(255),\n")
    f.write("    Rating DECIMAL(3,2),\n")
    f.write("    JumlahPengunjung INT,\n")
    f.write("    kode VARCHAR(255)\n");
    f.write(");\n\n")

    # Write the INSERT statements
    for statement in sql_statements:
        f.write(statement + "\n")

print("SQL file 'wisata.sql' created successfully.")

SQL file 'wisata.sql' created successfully.
